In [ ]:
import torch
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_features = train_data.iloc[:, 1:-1]
train_labels = train_data.iloc[:, -1]
test_features = test_data.iloc[:, 1:]
all_features = pd.concat([train_features, test_features])
all_features = all_features.apply(lambda x: (x - x.mean()) / (x.std()) if x.dtypes != 'object' else x)
train_labels = train_labels.apply(lambda x: (x - train_labels.mean()) / (train_labels.std()))
all_features = all_features.fillna(0)
all_features = pd.get_dummies(all_features, dummy_na=True)

train_features = torch.tensor(all_features.iloc[:train_data.shape[0],:].values, dtype=torch.float32)
test_features = torch.tensor(all_features.iloc[train_data.shape[0]:,:].values, dtype=torch.float32)

train_labels = torch.tensor(train_labels.values, dtype=torch.float32).reshape(-1,1)

def loss(y_hat, y):
    return torch.mean(torch.log(torch.abs(y_hat - y) + 1))

def train(train_iter, test_iter, net, loss, trainer, num_epochs=400, device=torch.device('cuda')):
    net.to(device)
    train_ls, test_ls = [], []
    for epoch in range(num_epochs):
        net.train()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            trainer.step()
        net.eval()
        with torch.no_grad():
            train_loss = sum([loss(net(X.to(device)), y.to(device)) for X, y in train_iter])/len(train_iter)
            test_loss = sum([loss(net(X.to(device)), y.to(device)) for X, y in test_iter])/len(test_iter)
        train_ls.append(train_loss)
        test_ls.append(test_loss)
    TrainLoss = sum(train_ls).item() / len(train_ls)
    TestLoss = sum(test_ls).item() / len(test_ls)
    return TrainLoss, TestLoss

def k_fold(k, train_features, train_labels, num_epochs=400, device=torch.device('cuda')):
    fold_size = train_features.shape[0] // k
    train_ls, test_ls = [], []
    for fold in range(k):
        net = torch.nn.Sequential(
        torch.nn.Linear(train_features.shape[1], 512),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.5),
        torch.nn.Linear(512, 256),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.5),
        torch.nn.Linear(256, 1)
        ).to(torch.device('cuda'))

        trainer = torch.optim.SGD(net.parameters(), lr=0.01, weight_decay=0.002)
        val_idx = slice(fold*fold_size, (fold + 1)*fold_size)
        train_idx = list(range(0, fold*fold_size)) + list(range((fold + 1)*fold_size, train_features.shape[0]))
        train_iter = DataLoader(TensorDataset(train_features[train_idx,:], train_labels[train_idx,:]), batch_size=128, shuffle=True)
        test_iter = DataLoader(TensorDataset(train_features[val_idx,:], train_labels[val_idx, :]), batch_size=128, shuffle=False)
        train_loss, test_loss = train(train_iter, test_iter, net, loss, trainer, num_epochs, device)
        print(f'Fold {fold + 1}, Train Loss: {train_loss}, Test Loss: {test_loss}')
        train_ls.append(train_loss)
        test_ls.append(test_loss)
    train_ls_mean = sum(train_ls) / k
    test_ls_mean = sum(test_ls) / k
    print(f'{k}-Fold Cross-Validation, Average Train Loss: {train_ls_mean}, Average Test Loss: {test_ls_mean}')
    return train_ls_mean, test_ls_mean

k_fold(5, train_features, train_labels, num_epochs=400, device=torch.device('cuda'))



        



Fold 1, Train Loss: 0.15596406936645507, Test Loss: 0.17457050323486328
Fold 2, Train Loss: 0.15667111396789551, Test Loss: 0.17668994903564453
Fold 3, Train Loss: 0.15647167205810547, Test Loss: 0.16977375030517577
Fold 4, Train Loss: 0.16230300903320313, Test Loss: 0.15621325492858887
Fold 5, Train Loss: 0.15479783058166505, Test Loss: 0.17842435836791992
5-Fold Cross-Validation, Average Train Loss: 0.15724153900146484, Average Test Loss: 0.17113436317443848


(0.15724153900146484, 0.17113436317443848)